.. _nb_results:

## Results

After an algorithm has been executed a result object is returned. In the following, single- and multi-objective runs with and without constraints are shown and the corresponding `Result` object is explained:

In [1]:
from pymoo.algorithms.so_genetic_algorithm import GA
from pymoo.factory import get_problem
from pymoo.optimize import minimize


problem = get_problem("sphere")
algorithm = GA(pop_size=5)
res = minimize(problem,
               algorithm,
               ('n_gen', 30),
               seed=1)


In this single-objective optimization problem, there exists a single best solution that was found. 
The result contains directly the best found values in the corresponding spaces. 

- `res.X` design space values are 
- `res.F` objective spaces values
- `res.G` constraint values
- `res.CV` aggregated constraint violation
- `res.algorithm` algorithm object
- `res.pop` final population object
- `res.history` history of algorithm object. (only if `save_history` has been enabled during the algorithm initialization)

Note, when the `minimize` function is called a deep copy of the algorithm object is created.
This ensures that two independent runs with the same algorithm and same random seed have the same results without any side effects.

The values from the final population can be extracted by using the `get` method. 
The population object is used internally, and store information for each individual. The `get`
method allows to return vectors or matrices based on the provided properties.


In [2]:
res.X, res.F, res.G, res.CV

(array([0.41919891, 0.47902208, 0.36089478, 0.5115848 , 0.45634212,
        0.54653289, 0.37113741, 0.40521917, 0.36873299, 0.52631462]),
 array([0.07403714]),
 None,
 array([0.]))

In [3]:
res.algorithm, res.pop

(<pymoo.algorithms.so_genetic_algorithm.GA at 0x11f794e10>,
 Population([<pymoo.model.individual.Individual object at 0x11f7a2550>,
            dtype=object))

The values from the final population can be extracted by using the `get` method. 
The population object is used internally, and store information for each individual. The `get`
method allows to return vectors or matrices based on the provided properties.

In [4]:
res.pop.get("X"), res.pop.get("F")

(array([[0.41919891, 0.47902208, 0.36089478, 0.5115848 , 0.45634212,
         0.54653289, 0.37113741, 0.40521917, 0.36873299, 0.52631462],
        [0.41919891, 0.51282797, 0.36089478, 0.51388238, 0.44440297,
         0.54653289, 0.37113741, 0.45001996, 0.33100837, 0.51291793],
        [0.41919891, 0.50077623, 0.36089478, 0.51328358, 0.61967624,
         0.54653289, 0.37113741, 0.40521917, 0.37235219, 0.53021809],
        [0.41919891, 0.51282797, 0.36089478, 0.51388238, 0.64152746,
         0.54653289, 0.37113741, 0.40521917, 0.37502487, 0.51449625],
        [0.41919891, 0.50077623, 0.36089478, 0.51328358, 0.42256495,
         0.54653289, 0.37113741, 0.40521917, 0.32833565, 0.52863977]]),
 array([[0.07403714],
        [0.07932131],
        [0.08533991],
        [0.08984959],
        [0.09009549]]))

In this run, the problem did not have any constraints and therefore `res.G` evaluated to `None`.
Also note, that `res.CV` will always be set to 0 no matter if the problem has constraints or not.

Let us consider a problem that has, in fact, constraints:

In [5]:
problem = get_problem("g01")
algorithm = GA(pop_size=5)
res = minimize(problem,
               algorithm,
               ('n_gen', 5),
               verbose=True,
               seed=1)

n_gen |  n_eval |   cv (min)   |   cv (avg)   |     favg     |     fopt    
    1 |       5 |  1.35027E+02 |  5.47570E+02 |            - |            -
    2 |      10 |  1.35027E+02 |  3.60756E+02 |            - |            -
    3 |      15 |  1.35027E+02 |  2.25988E+02 |            - |            -
    4 |      20 |  1.35027E+02 |  1.44347E+02 |            - |            -
    5 |      25 |  1.35009E+02 |  1.35709E+02 |            - |            -


In [6]:
res.X, res.F, res.G, res.CV

(None, None, None, None)

Here, the algorithm was not able to find any feasible solution in 5 generations. Therefore, all values contained in the results are equals to `None`. If in this case the least feasible solution should be returned, the flag `return_least_infeasible` needs to be enabled:

In [7]:
problem = get_problem("g01")
algorithm = GA(pop_size=5, return_least_infeasible=True)
res = minimize(problem,
               algorithm,
               ('n_gen', 5),
               verbose=True,
               seed=1)

n_gen |  n_eval |   cv (min)   |   cv (avg)   |     favg     |     fopt    
    1 |       5 |  1.35027E+02 |  5.47570E+02 |            - |            -
    2 |      10 |  1.35027E+02 |  3.60756E+02 |            - |            -
    3 |      15 |  1.35027E+02 |  2.25988E+02 |            - |            -
    4 |      20 |  1.35027E+02 |  1.44347E+02 |            - |            -
    5 |      25 |  1.35009E+02 |  1.35709E+02 |            - |            -


In [8]:
res.X, res.F, res.G, res.CV

(array([ 0.75014431,  0.99342958,  0.74816565,  0.28044399,  0.78927933,
         0.10322601,  0.44789353,  0.9085955 ,  0.29361415, 28.77753386,
        13.00285721,  1.93669579,  0.67883553]),
 array([-44.01771073]),
 array([35.26753887, 23.71084958,  8.42274348, 22.77637934,  5.05542053,
        -4.04862945, 27.42736655, 12.34851167, -0.17410937]),
 array([135.00881002]))

We have made this design decision, because an infeasible solution can often not be considered as a solution
of the optimization problem. Therefore, having a solution equals to `None` indicates the fact no feasible solution has been found.

If the problem has multiple objective, the result object has the same structure but `res.X`, `res.F`, `res.G`, `res.CV` are a set 
of non-dominated solutions instead of a single one.

In [9]:
from pymoo.algorithms.nsga2 import NSGA2

problem = get_problem("zdt2")
algorithm = NSGA2()
res = minimize(problem,
               algorithm,
               ('n_gen', 10),
               seed=1)

In [10]:
res.F

array([[9.20203865e-01, 2.78196295e+00],
       [8.44885722e-01, 3.00931642e+00],
       [3.22198205e-04, 3.29352972e+00],
       [2.31557361e-05, 3.58881094e+00],
       [5.05496673e-04, 3.28432654e+00],
       [1.46697798e-02, 3.14288874e+00],
       [2.61436263e-01, 3.11387760e+00],
       [1.37675126e-02, 3.18163525e+00]])